# Importing Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Data Processing of Dataset 1

In [4]:
train_df = pd.read_csv('curebridge_training_balanced.csv')
train_df

,text,label
0,@ZubearSays Any real nigga isn't letting this ...,Racial Discrimination
1,@MoradoSkittle @prolifejewess @DAConsult @Kell...,Respectful
2,"the only thing i wish, i wish a nigga would",Racial Discrimination
3,You saudias are not friends of Muslim idiots c...,Religious Discrimination
4,@JaydenT2399 @TractorLaw @holmes_gael @erconge...,Religious Discrimination
...,...,...
99985,Thank fuck they didn’t dance remix Peru #LoveI...,Respectful
99986,Sierra Canyon vs. Alemany - California High Sc...,Respectful
99987,@queenbellaxoxo @PontiacMadeDDG @HalleBailey B...,Gender-Based Harassment
99988,What you like yo nigga to call you,Racial Discrimination


In [5]:
train_df['label'].value_counts()

,count
label,
Respectful,50000
Racial Discrimination,17000
Gender-Based Harassment,17000
Religious Discrimination,15990


In [6]:
train_df.drop_duplicates(subset = ['text'], inplace = True)
train_df.duplicated().sum()

np.int64(0)

# Data Processing of Dataset 2

In [7]:
eval_df = pd.read_csv('curebridge_tweets_evaluation.csv')
eval_df.rename(columns = {'tweet_text' : 'text', 'conversation_label' : 'label'}, inplace = True)
eval_df

,text,label
0,"In other words #katandandre, your food was cra...",Respectful
1,Why is #aussietv so white? #MKR #theblock #ImA...,Respectful
2,@XochitlSuckkks a classy whore? Or more red ve...,Respectful
3,"@Jason_Gio meh. :P thanks for the heads up, b...",Respectful
4,@RudhoeEnglish This is an ISIS account pretend...,Respectful
...,...,...
47687,"Black ppl aren't expected to do anything, depe...",ethnicity
47688,Turner did not withhold his disappointment. Tu...,ethnicity
47689,I swear to God. This dumb nigger bitch. I have...,ethnicity
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity


In [8]:
eval_df['label'].value_counts()

,count
label,
Religious Discrimination,7998
age,7992
gender,7973
ethnicity,7961
Respectful,7945
other_cyberbullying,7823


In [9]:
eval_df.drop_duplicates(subset = ['text'], inplace = True)
eval_df.duplicated().sum()

np.int64(0)

# Splitting Data

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Assuming cyber1 is your DataFrame with columns 'text' and 'label'
texts = train_df['text']
labels = train_df['label']

# Encode the labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
label_mapping = {original_label: int_label for original_label, int_label in zip(train_df['label'], encoded_labels)}
print(label_mapping)
# Split the data into training and testing sets (80-20 split)
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, encoded_labels, test_size=0.2, random_state=42)

# Split the training data into training and validation sets (80-20 split)
train_texts, valid_texts, train_labels, valid_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42)

{'Racial Discrimination': np.int64(1), 'Respectful': np.int64(3), 'Religious Discrimination': np.int64(2), 'Gender-Based Harassment': np.int64(0)}


In [11]:
!nvidia-smi

Mon May 19 01:43:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Importing and Fine Tuning the Model

In [12]:
from transformers import XLNetTokenizer, XLNetForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from torch.utils.data import DataLoader, Dataset
import torch
import numpy as np

# ----------------------------- #
# Metric Computation Function
# ----------------------------- #
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_score(labels, predictions)
    class_report = classification_report(labels, predictions, output_dict=True)
    conf_matrix = confusion_matrix(labels, predictions).tolist()  # For JSON serialization

    return {
        'accuracy': accuracy,
        'classification_report': class_report,
        'confusion_matrix': conf_matrix
    }

# ----------------------------- #
# Load Tokenizer
# ----------------------------- #
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

# ----------------------------- #
# Tokenization Function
# ----------------------------- #
def tokenize_data(texts, labels):
    encodings = tokenizer(texts, truncation=True, padding=True)
    return {
        "input_ids": encodings['input_ids'],
        "attention_mask": encodings['attention_mask'],
        "labels": labels
    }

# train_texts, train_labels, valid_texts, valid_labels

train_data = tokenize_data(list(train_texts), train_labels)
valid_data = tokenize_data(list(valid_texts), valid_labels)

# ----------------------------- #
# Dataset Class
# ----------------------------- #
class CureBridgeDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings['input_ids'])

train_dataset = CureBridgeDataset(train_data)
valid_dataset = CureBridgeDataset(valid_data)

# ----------------------------- #
# Load XLNet Model
# ----------------------------- #
model = XLNetForSequenceClassification.from_pretrained(
    'xlnet-base-cased',
    num_labels=len(set(labels))  # 🔁 Make sure `labels` is your full label list
)

# ----------------------------- #
# Training Arguments
# ----------------------------- #
training_args = TrainingArguments(
    output_dir='./curebridge_model_output',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    logging_dir='./curebridge_logs',
    report_to=[],  # disables logging to external services
    fp16=True
)

# ----------------------------- #
# Hugging Face Trainer
# ----------------------------- #
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

# ----------------------------- #
# Train & Save Model
# ----------------------------- #
trainer.train()

save_directory = "./curebridge_xlnet_classifier"
trainer.save_model(save_directory)
tokenizer.save_pretrained(save_directory)

print(f"XLNet model and tokenizer saved to {save_directory}")


spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/467M [00:00<?, ?B/s]

Step,Training Loss
500,0.247500
1000,0.069000
1500,0.058500
2000,0.046100
2500,0.039500
3000,0.042900
3500,0.032000
4000,0.026100


XLNet model and tokenizer saved to ./curebridge_xlnet_classifier


In [ ]:
pip freeze > requirements.txt

# Testing the Model

In [14]:
# ✅ Convert test texts to list and tokenize
test_texts = list(test_texts)
test_data = tokenize_data(test_texts, test_labels)

# ✅ Use the same dataset class used for training
test_dataset = CureBridgeDataset(test_data)

# ✅ Evaluate the model using Hugging Face Trainer
evaluation_results = trainer.evaluate(test_dataset)

# ✅ Print results
print("🧪 CureBridge Test Evaluation Results:", evaluation_results)


🧪 CureBridge Test Evaluation Results: {'eval_loss': 0.02323167212307453, 'eval_accuracy': 0.9957995799579958, 'eval_classification_report': {'0': {'precision': 0.9982164090368609, 'recall': 0.993491124260355, 'f1-score': 0.9958481613285883, 'support': 3380.0}, '1': {'precision': 0.9928571428571429, 'recall': 0.9976076555023924, 'f1-score': 0.9952267303102625, 'support': 3344.0}, '2': {'precision': 0.9948914431673053, 'recall': 0.9892063492063492, 'f1-score': 0.9920407513530722, 'support': 3150.0}, '3': {'precision': 0.9962532044961546, 'recall': 0.9980244962465429, 'f1-score': 0.9971380637520971, 'support': 10124.0}, 'accuracy': 0.9957995799579958, 'macro avg': {'precision': 0.9955545498893659, 'recall': 0.9945824063039098, 'f1-score': 0.9950634266860051, 'support': 19998.0}, 'weighted avg': {'precision': 0.995802642091957, 'recall': 0.9957995799579958, 'f1-score': 0.9957975345352814, 'support': 19998.0}}, 'eval_confusion_matrix': [[3358, 17, 2, 3], [2, 3336, 2, 4], [2, 1, 3116, 31], [

# Visualizing the results

In [17]:
import plotly.graph_objects as go
import numpy as np

# Your confusion matrix
conf_matrix = np.array([[3333, 2, 4, 5], [17, 3362, 0, 1], [16, 7, 10074, 27], [3, 3, 26, 3118]])
class_labels = ['Racial Discrimination', 'Gender-Based Harassment', 'Respectful', 'Religious Discrimination']

# Transpose for grouped bars
bars = []
for i, pred_label in enumerate(class_labels):
    bars.append(go.Bar(name=f"Predicted: {pred_label}", x=class_labels, y=conf_matrix[:, i]))

fig = go.Figure(data=bars)
fig.update_layout(
    barmode='group',
    title='Confusion Matrix (Grouped Bar Chart)',
    xaxis_title='True Label',
    yaxis_title='Count',
    legend_title='Predicted Label'
)
fig.show()


![newplot%20%281%29.png](attachment:newplot%20%281%29.png)

In [22]:
import torch
import numpy as np
from sklearn.metrics import confusion_matrix

sample_conversation = ['''Doctor: Hello, I’m Dr. Rahman. How can I assist you today?

Patient: I was hoping to see a real doctor, not someone from your background.

Doctor: I assure you, I’m fully qualified and here to help. What symptoms are you experiencing?

Patient: Honestly, I don’t feel comfortable being treated by someone who doesn’t even belong here.

Doctor: I want to focus on your care and ensure you’re healthy. Please let me know how you’re feeling.

Patient: People like you should stick to your own communities. I want someone who understands real medicine.

Doctor: I’m going to have to end this consultation if this disrespect continues. Discrimination has no place in healthcare.

Patient: Whatever. I’ll go somewhere else. I don’t trust your kind with my health.

Doctor: This conversation will be documented and reported. I hope you reflect on the harm such comments cause.
''']

encoded = tokenizer(sample_conversation, truncation=True, padding=True)
example_encodings = {
    "input_ids": encoded["input_ids"],
    "attention_mask": encoded["attention_mask"],
    "labels": [0]  # Dummy label
}

example_dataset = CureBridgeDataset(example_encodings)

prediction_output = trainer.predict(example_dataset)

predicted_label_idx = np.argmax(prediction_output.predictions, axis=-1)[0]
confidence = torch.softmax(torch.tensor(prediction_output.predictions[0]), dim=-1)[predicted_label_idx].item()

# label mapping
label_mapping = {
    0: "Racial Discrimination",
    1: "Gender-Based Harassment",
    2: "Respectful",
    3: "Religious Discrimination"
}

predicted_label = label_mapping[predicted_label_idx]

print(f"Predicted Label: {predicted_label}")
print(f"Confidence Score: {confidence:.2f}")

Predicted Label: Religious Discrimination
Confidence Score: 0.99
